# Agents and Agents Runtime

In [10]:
from typing import Callable
from pydantic import BaseModel
from autogen_core import AgentId, SingleThreadedAgentRuntime


from autogen_core import (
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    default_subscription,
    message_handler,
)

`Message` defines the model message that is passed between the agents.
`Modifier` and `Checker` are the agent classes; each defines its procedure for handling messages: in this example, `Modifier` decrements a counter and publish a response, while `Checker` reads Modifier's message and checks if the generated is higher than 1, otherwise stops sending messages back.


**NB**: agents’ logic, whether it is using model or code executor, is completely decoupled from how messages are delivered -> the agents are responsible for their own logic

In [15]:
class Message(BaseModel):
    content: int


@default_subscription
class Modifier(RoutedAgent):
    def __init__(self, modify_val: Callable[[int], int]) -> None:
        super().__init__("A modifier agent.")
        self._modify_val = modify_val

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        val = self._modify_val(message.content)
        print(f"{'-'*80}\nModifier:\nModified {message.content} to {val}")
        await self.publish_message(Message(content=val), DefaultTopicId())  # type: ignore


@default_subscription
class Checker(RoutedAgent):
    def __init__(self, run_until: Callable[[int], bool]) -> None:
        super().__init__("A checker agent.")
        self._run_until = run_until

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        if not self._run_until(message.content):
            print(f"{'-'*80}\nChecker:\n{message.content} passed the check, continue.")
            await self.publish_message(
                Message(content=message.content), DefaultTopicId()
            )
        else:
            print(f"{'-'*80}\nChecker:\n{message.content} failed the check, stopping.")

**Core idea**: while agents are responsible for their own logic, the framework is responsible for providing a communication infrastructure, We call the communication infrastructure an **AGENT RUNTIME**.

Agent runtime is a key concept: besides delivering messages, it also manages agents’ lifecycle. So the creation of agents are handled by the runtime itself.

Here we use *SingleThreadedAgentRuntime*, a local embedded agent runtime implementation.


In [16]:
from enum import StrEnum, auto


# agent types (indicates each agent "family" -> different from agent's class)
class AgentType(StrEnum):
    MODIFIER: str = auto()
    CHECKER: str = auto()


# Create an local embedded runtime.
runtime = SingleThreadedAgentRuntime()

# Register the modifier and checker agents by providing
# their agent types, the factory functions for creating instance and subscriptions.
await Modifier.register(
    runtime,
    AgentType.MODIFIER,
    # Modify the value by subtracting 1
    lambda: Modifier(modify_val=lambda x: x - 1),
)

await Checker.register(
    runtime,
    AgentType.CHECKER,
    # Run until the value is less than or equal to 1
    lambda: Checker(run_until=lambda x: x <= 1),
)

In [17]:
# Start the runtime and send a direct message to the checker.
runtime.start()
await runtime.send_message(Message(content=10), AgentId("checker", "default"))
await runtime.stop_when_idle()